In [1]:
import requests
import pandas as pd

BASE_URL = "https://fantasy.premierleague.com/api"

bootstrap = requests.get(f"{BASE_URL}/bootstrap-static/").json()

players = {p["id"]: p for p in bootstrap["elements"]}
teams_lookup = {t["id"]: t["name"] for t in bootstrap["teams"]}

In [2]:
def get_formation(picks):
    """
    Determine FPL team formation from picks.
    Only consider the starting 11 (position 1–11).
    """
    formation = {"DEF": 0, "MID": 0, "FWD": 0}

    for pick in picks["picks"]:
        if pick["position"] <= 11:  # starters only - to counter bboost
            if pick["element_type"] == 2:  # DEF
                formation["DEF"] += 1
            elif pick["element_type"] == 3:  # MID
                formation["MID"] += 1
            elif pick["element_type"] == 4:  # FWD
                formation["FWD"] += 1

    return f"{formation['DEF']}-{formation['MID']}-{formation['FWD']}"

def get_full_league_standings(league_id: int):
    standings = []
    page = 1
    while True:
        url = f"{BASE_URL}/leagues-classic/{league_id}/standings/?page_standings={page}"
        resp = requests.get(url).json()
        
        results = resp["standings"]["results"]
        standings.extend(results)
        
        if resp["standings"]["has_next"]:
            page += 1
        else:
            break
    
    return standings

In [3]:
league_id = 815838
gw = 1

# Fetch league standings
league = get_full_league_standings(league_id)
entries = [team["entry"] for team in league]

teams_data = []
all_picks = []

for entry_id in entries:
    entry_info = requests.get(f"{BASE_URL}/entry/{entry_id}/").json()
    picks = requests.get(f"{BASE_URL}/entry/{entry_id}/event/{gw}/picks/").json()
    all_picks.append(picks)
    transfers = requests.get(f"{BASE_URL}/entry/{entry_id}/transfers/").json()
    
    # Transfers of recent GW
    gw_transfers = [
        (players[t["element_in"]]["web_name"], 
         players[t["element_out"]]["web_name"], 
         players[t["element_in"]]["total_points"] - players[t["element_out"]]["total_points"])
        for t in transfers if t["event"] == gw
    ]
    
    # Formation
    formation = get_formation(picks)
    
    # Captain & VC
    captain = players[next(p["element"] for p in picks["picks"] if p["is_captain"])]["web_name"]
    vice_captain = players[next(p["element"] for p in picks["picks"] if p["is_vice_captain"])]["web_name"]

    playing_XI = ", ".join(players[p["element"]]["web_name"] 
                           for p in picks['picks'] if p["multiplier"] != 0)
    bench = ", ".join(players[p["element"]]["web_name"] 
                           for p in picks['picks'] if p["multiplier"] == 0)
    
    teams_data.append({
        "Team name": entry_info["name"],
        "Total points": entry_info["summary_overall_points"],
        "GW points": entry_info["summary_event_points"],
        "Transfers": gw_transfers,
        "Formation": formation,
        "Captain": captain,
        "Vice Captain": vice_captain,
        "Playing XI": playing_XI,
        "Bench": bench,
        "Chips used": picks.get("active_chip")
    })

df_teams = pd.DataFrame(teams_data)

In [4]:
df_teams

,Team name,Total points,GW points,Transfers,Formation,Captain,Vice Captain,Playing XI,Bench,Chips used
0,Hustle & Flo,78,78,[],3-4-3,Haaland,Wirtz,"Raya, Andersen, Aït-Nouri, Muñoz, Wirtz, Palme...","Areola, Baleba, Alex Moreno, Tuanzebe",None
1,K8 the Gr8,77,77,[],4-4-2,M.Salah,Wood,"Pickford, Schär, Gabriel, Tarkowski, Aina, Cun...","Verbruggen, Ndiaye, Alex Moreno, Kostoulas",None
2,IwobiKlose,60,60,[],3-5-2,M.Salah,Wirtz,"Verbruggen, Frimpong, Murillo, Tarkowski, Wirt...","Dúbravka, Piroe, Gudmundsson, Reinildo",None
3,TheatreOfMemes,55,55,[],4-4-2,M.Salah,Palmer,"Sánchez, Van de Ven, Wan-Bissaka, Konsa, Frimp...","Dúbravka, Estève, A.Ramsey, Marc Guiu",None
4,Saka Potatoes,53,53,[],4-4-2,Delap,Pickford,"Pickford, James, Van de Ven, Wan-Bissaka, Rome...",,bboost
5,All Change,51,51,[],3-4-3,Palmer,João Pedro,"Sánchez, J.Timber, Frimpong, Pedro Porro, Palm...","Dúbravka, Konsa, Malen, Wan-Bissaka",None
6,EZe-pass,49,49,[],4-4-2,Watkins,Palmer,"Dúbravka, Murillo, Tarkowski, Virgil, Pedro Po...","Areola, Estève, Rice, Barnes",None
7,Gameofthrowins,49,49,[],4-4-2,M.Salah,Palmer,"Sánchez, Virgil, Wan-Bissaka, Murillo, Tarkows...","Dúbravka, Hill, Marc Guiu, G.Rodriguez",None
8,Deutschmeister,48,48,[],3-5-2,M.Salah,Palmer,"Sánchez, Murillo, Pedro Porro, Andersen, M.Sal...","Dúbravka, Dorgu, Estève, Marc Guiu",None
9,Elland Roadie,44,44,[],4-4-2,Bowen,Aït-Nouri,"Sels, Cucurella, Kerkez, Pedro Porro, Aït-Nour...","Sánchez, J.Timber, Cunha, Raúl",None


In [5]:
from collections import defaultdict

picks_count = defaultdict(int)
captain_count = defaultdict(int)

for team in all_picks:
    active_picks = {p["element"]: p for p in team["picks"]}

    # Determine who was captain + vice
    cap_id = next(p["element"] for p in team["picks"] if p["is_captain"])
    vice_id = next(p["element"] for p in team["picks"] if p["is_vice_captain"])

    # Fetch autosubs info (to check if captain missed out)
    autosubs = team.get("automatic_subs", [])

    # Count all picks
    for p in active_picks.values():
        picks_count[p["element"]] += 1

    # If captain is autosubbed out, vice takes over
    cap_out = any(s["element_out"] == cap_id for s in autosubs)
    if cap_out:
        vice_out = any(s["element_out"] == vice_id for s in autosubs)
        if vice_out: #Case where both C and VC don't play
            None
        else:
            captain_count[vice_id] += 1
    else:
        captain_count[cap_id] += 1

# ===== 5. Build DataFrame =====
footballers_data = []
for p in bootstrap["elements"]:
    footballers_data.append({
        "Player ID":p["id"] ,
        "Footballer name": p["web_name"],
        "Total points": p["total_points"],
        "GW points": p["event_points"],
        "Real team name": teams_lookup[p["team"]],
        "Real team ID": p["team"],
        "Price (in Millions £)": p["now_cost"] / 10,
        "Price last GW (in Millions £)": p["cost_change_event"] / 10 + (p["now_cost"] / 10),
        "Price difference (in Millions £)": p["cost_change_event"] / 10,
        "Times chosen in squad": picks_count[p["id"]],
        "Times captained": captain_count[p["id"]]
    })

df_footballers = pd.DataFrame(footballers_data)

In [6]:
df_footballers.iloc[440:490]

,Player ID,Footballer name,Total points,GW points,Real team name,Real team ID,Price (in Millions £),Price last GW (in Millions £),Price difference (in Millions £),Times chosen in squad,Times captained
440,422,Kovačić,0,0,Man City,13,6.0,6.0,0.0,0,0
441,423,N.Gonzalez,3,3,Man City,13,6.0,6.0,0.0,0,0
442,424,Bobb,6,6,Man City,13,5.5,5.5,0.0,0,0
443,425,Echeverri,0,0,Man City,13,5.5,5.5,0.0,0,0
444,427,Reijnders,10,10,Man City,13,5.6,5.7,0.1,1,0
445,428,Nypan,0,0,Man City,13,5.0,5.0,0.0,0,0
446,429,Phillips,0,0,Man City,13,5.0,5.0,0.0,0,0
447,430,Haaland,13,13,Man City,13,14.0,14.0,0.0,1,1
448,119,Mbeumo,2,2,Man Utd,14,8.0,8.0,0.0,0,0
449,431,Bayindir,2,2,Man Utd,14,5.0,5.0,0.0,0,0


------------ Metrics -----------

a) Rules based

In [7]:
rule_based_metrics = []

#1
league_topper = f"{df_teams["Team name"].iloc[0]} topped the league"
rule_based_metrics.append(league_topper)

#2
league_bottom = f"{df_teams["Team name"].iloc[-1]} is at the bottom"
rule_based_metrics.append(league_bottom)

#4
idx_max_change = df_teams["GW points"].abs().idxmax()
value_max_change = int(df_teams.iloc[idx_max_change]["GW points"])
team_max_change = df_teams.iloc[idx_max_change]["Team name"]
max_change_in_points = f"{team_max_change} showed the maximum change of points over the last week. Their change: {value_max_change} points from {int(df_teams.iloc[idx_max_change]["Total points"]) - value_max_change} to {int(df_teams.iloc[idx_max_change]["Total points"])}."
rule_based_metrics.append(max_change_in_points)

#5
idx_most_points_gained = df_teams["GW points"].idxmax()
value_most_points_gained = df_teams.iloc[idx_most_points_gained]["GW points"]
team_most_points_gained = df_teams.iloc[idx_most_points_gained]["Team name"]
most_points_gained = f"{team_most_points_gained} gained the most points ({value_most_points_gained}) this Gameweek."
rule_based_metrics.append(most_points_gained)

#6
idx_least_points_gained = df_teams["GW points"].idxmin()
value_least_points_gained = df_teams.iloc[idx_least_points_gained]["GW points"]
team_least_points_gained = df_teams.iloc[idx_least_points_gained]["Team name"]
least_points_gained = f"{team_least_points_gained} gained the least points ({value_least_points_gained}) this Gameweek."
rule_based_metrics.append(least_points_gained)

#7
idx_highest_scoring_footballer = df_footballers["GW points"].idxmax()
value_highest_scoring_footballer = df_footballers.iloc[idx_highest_scoring_footballer]["GW points"]
name_highest_scoring_footballer = df_footballers.iloc[idx_highest_scoring_footballer]["Footballer name"]
highest_scoring_footballer = f"Footballer {name_highest_scoring_footballer} gained the most points ({value_highest_scoring_footballer}) this Gameweek."
rule_based_metrics.append(highest_scoring_footballer)

#8
idx_lowest_scoring_footballer = df_footballers["GW points"].idxmin()
value_lowest_scoring_footballer = df_footballers.iloc[idx_lowest_scoring_footballer]["GW points"]
name_lowest_scoring_footballer = df_footballers.iloc[idx_lowest_scoring_footballer]["Footballer name"]
lowest_scoring_footballer = f"Footballer {name_lowest_scoring_footballer} gained the least points ({value_lowest_scoring_footballer}) this Gameweek."
rule_based_metrics.append(lowest_scoring_footballer)

#9
team_scores = {}
for team_id in range(1,21):
    team_scores[team_id] = sum(df_footballers[df_footballers["Real team ID"] == team_id]["GW points"])

max_team = max(team_scores, key=team_scores.get)
max_score = team_scores[max_team]
highest_scoring_real_team = f"{teams_lookup[max_team]} players had the highest score combined: {max_score} points."
rule_based_metrics.append(highest_scoring_real_team)

#10
min_team = min(team_scores, key=team_scores.get)
min_score = team_scores[min_team]
lowest_scoring_real_team = f"{teams_lookup[min_team]} players had the lowest score combined: {min_score} points."
rule_based_metrics.append(lowest_scoring_real_team)

#11
max_tuple = None
max_value = float("-inf")
max_row = None
best_transfer = None

min_tuple = None
min_value = float("inf")
min_row = None
worst_transfer = None

for idx, row in df_teams.iterrows():
    for tup in row["Transfers"]:
        x = tup[2]  # third element
        if x > max_value:
            max_value = x
            max_tuple = tup
            max_row = idx
        if x < min_value:
            min_value = x
            min_tuple = tup
            min_row = idx

if max_row:
    team_best_transfer = df_teams.iloc[max_row]["Team name"]
    if max_tuple:
        best_transfer = f"{team_best_transfer} got {max_tuple[0]} in and removed {max_tuple[1]} and saw a change of {max_value} points."
rule_based_metrics.append(best_transfer)

#12
if min_row:
    team_worst_transfer = df_teams.iloc[min_row]["Team name"]
    if min_tuple:
        worst_transfer = f"{team_worst_transfer} got {min_tuple[0]} in and removed {min_tuple[1]} and saw a change of {min_value} points."
rule_based_metrics.append(worst_transfer)

#13
highest_price_increase = None
df_footballers["Price difference (in Millions £)"]

idx_highest_increased_price = df_footballers["Price difference (in Millions £)"].idxmax()
value_highest_increased_price = df_footballers.iloc[idx_highest_increased_price]["Price difference (in Millions £)"]
name_highest_increased_price = df_footballers.iloc[idx_highest_increased_price]["Footballer name"]
highest_increased_price = f"{name_highest_increased_price} gained the most value in the market: (£{value_highest_increased_price}M) this Gameweek."
rule_based_metrics.append(highest_increased_price)

#14
most_expensive_players = df_footballers.sort_values("Price (in Millions £)", ascending=False).head(5)

price_vs_points = "Highest priced footballers \n"
for idx, row in most_expensive_players.iterrows():
    if players[row["Player ID"]].get("minutes") == 0:
        continue
    price_vs_points += f"{row["Footballer name"]} valued at {row["Price (in Millions £)"]} scored {row["GW points"]} points. \n"
rule_based_metrics.append(price_vs_points)

#15
most_chosen_players = df_footballers.sort_values("Times chosen in squad", ascending=False).head(5)

chosen_vs_points = "Most picked footballers \n"
for idx, row in most_chosen_players.iterrows():
    if players[row["Player ID"]].get("minutes") == 0:
        continue
    chosen_vs_points += f"{row["Footballer name"]} chosen {row["Times chosen in squad"]} times scored {row["GW points"]} points. \n"
rule_based_metrics.append(chosen_vs_points)

#16
most_captained_players = df_footballers.sort_values("Times captained", ascending=False).head(5)

captained_vs_points = "Most times picked as captains \n"
for idx, row in most_captained_players.iterrows():
    if players[row["Player ID"]].get("minutes") == 0:
        continue
    captained_vs_points += f"{row["Footballer name"]} chosen captain {row["Times captained"]} times scored {row["GW points"]} points. \n"
rule_based_metrics.append(captained_vs_points)

#18
chips_usage = ""
for idx, row in df_teams.iterrows():
    if row["Chips used"]:
        chips_usage += f"{row["Team name"]} used the chip(s): {row["Chips used"]}"
rule_based_metrics.append(chips_usage)

In [8]:
rule_based_metrics

['Hustle & Flo topped the league',
 'Ha-Cunha Mateta is at the bottom',
 'Hustle & Flo showed the maximum change of points over the last week. Their change: 78 points from 0 to 78.',
 'Hustle & Flo gained the most points (78) this Gameweek.',
 'People on the pitch gained the least points (40) this Gameweek.',
 'Footballer Ballard gained the most points (17) this Gameweek.',
 'Footballer Doherty gained the least points (-1) this Gameweek.',
 'Man City players had the highest score combined: 86 points.',
 'West Ham players had the lowest score combined: 18 points.',
 None,
 None,
 'Semenyo gained the most value in the market: (£0.1M) this Gameweek.',
 'Highest priced footballers \nM.Salah valued at 14.5 scored 8 points. \nHaaland valued at 14.0 scored 13 points. \nPalmer valued at 10.5 scored 3 points. \nSaka valued at 10.0 scored 3 points. \n',
 'Most picked footballers \nPalmer chosen 11 times scored 3 points. \nDúbravka chosen 9 times scored 2 points. \nWirtz chosen 8 times scored 2 p

b) LLM based

In [9]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [10]:
rule_based_metrics_text = "\n".join(f"- {metric}" for metric in rule_based_metrics)
df_teams_text = df_teams.to_string(index=False)

In [11]:
prompt = f"""You have to write a commissioner-style roundup for our Fantasy Premier League private league Gameweek 1. I am 
providing you some highlight-metrics that I could find from my data. I am also giving you the standings table of my league which 
you need to observe and make comments about. You also need to have knowledge of the Premier Leagure footballing world and make 
remarks that the fans can relate to, as a part of your sense of humor. 

Here are some of the highlights: {rule_based_metrics_text}
Mention most of these stats in the summary, depending on how interesting each is. 

Here is a teams table. Use it to comment on the standings of the teams, and how competitive what part of the table is {df_teams_text}
Using the table, also look for unusual stuff, or blunders that the players might have made for example not setting up a captain 
or vice-captain, playing injured/suspended players or the ones in transfer talks and not available for selection.

Use the metrics and the table provided to generate a commissioner styled FPL roundup for this GameWeek. Feel free to be creative!
"""

In [12]:
print(prompt)

You have to write a commissioner-style roundup for our Fantasy Premier League private league Gameweek 1. I am 
providing you some highlight-metrics that I could find from my data. I am also giving you the standings table of my league which 
you need to observe and make comments about. You also need to have knowledge of the Premier Leagure footballing world and make 
remarks that the fans can relate to, as a part of your sense of humor. 

Here are some of the highlights: - Hustle & Flo topped the league
- Ha-Cunha Mateta is at the bottom
- Hustle & Flo showed the maximum change of points over the last week. Their change: 78 points from 0 to 78.
- Hustle & Flo gained the most points (78) this Gameweek.
- People on the pitch gained the least points (40) this Gameweek.
- Footballer Ballard gained the most points (17) this Gameweek.
- Footballer Doherty gained the least points (-1) this Gameweek.
- Man City players had the highest score combined: 86 points.
- West Ham players had the lowest

In [13]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a witty fantasy football commissioner."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

print(response.choices[0].message.content)

Welcome, esteemed managers, to the glorious return of the Fantasy Premier League saga! Gameweek 1 is in the books, and just like a freshly promoted side, our league is bursting with optimism and questionable tactics. Let’s dive into the drama, shall we?

### Top of the League: Hustle & Flo
Ladies and gentlemen, give it up for Hustle & Flo, who came charging out of the gates like Erling Haaland on a particularly good hair day! A staggering 78 points—yes, you read that right—catapulted them to the top of the table. Captain Haaland delivered 13 points, proving that some things in life (like Man City winning and Haaland scoring) are as reliable as a James Milner cross.

### The Bottom Dwellers: Ha-Cunha Mateta
Meanwhile, Ha-Cunha Mateta finds themselves languishing at the bottom, with a paltry 40 points. Perhaps they were too busy pondering the mysteries of the universe, like why anyone would captain Palmer, who scored just 3 points.

### The Biggest Movers & Shakers
Hustle & Flo also cloc